In [1]:
#!pip install pyyaml==5.1
import torch
import torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

1.9.0+cu111 True
gcc (Gentoo 11.2.0 p1) 11.2.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
#%matplotlib notebook
import glob
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import Image
import time
import shutil
from os import path

from shapely.geometry import Polygon

# importing required libraries
import pandas as pd
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import pandas as pd
from ..utils.get_iou import get_max_iou, get_iou, get_overlap
import pickle
import random
import json
import os
from collections import Counter
import cv2
import numpy as np

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2 import model_zoo
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
from detectron2.engine import DefaultTrainer

In [7]:
from ..utils.metric_tablebank_my_implementation import metric_table_bank_union
from get_lines import get_relevant_lines

# Get data

In [4]:
path_data = "/data/rali5/Tmp/yockelle/TableBank/TableBank/Detection/"

In [10]:
from ..utils.get_models_and_set import get_train_data_word, get_train_data_latex, get_train_data_publaynet, get_train_data_word_pln, get_train_data_latex_pln, \
                                    get_train_data_pln_and_empty, get_train_data_and_word_pln_empty, get_train_data_and_latex_pln_empty, get_train_data_word_and_latex_pln_empty, \
                                    get_test_data_word, get_valid_data, get_train_data_latex, get_train_data_word, get_test_data_latex, get_test_data_publaynet

ImportError: attempted relative import with no known parent package

In [5]:
def get_valid_data():
    latex = pickle.load( open(path_data + "annotations/COCO_tablebank_latex_val.pickle", "rb" ) )
    word = pickle.load( open(path_data + "annotations/COCO_tablebank_word_val.pickle", "rb" ) )
    return latex + word
    
# TRAINING DATA
def get_train_data_latex():
    latex = pickle.load( open(path_data + "annotations/COCO_tablebank_latex_train.pickle", "rb" ) )
    return latex

def get_train_data_word():
    word = pickle.load( open(path_data + "annotations/COCO_tablebank_word_train.pickle", "rb" ) )
    return word

In [6]:
print("word", len(get_train_data_word()))
print("latex", len(get_train_data_latex()))
print("publaynet", len(get_train_data_publaynet()))
print("word publaynet", len(get_train_data_word_pln()))
print("latex publaynet", len(get_train_data_latex_pln()))
print("no table publaynet", len(get_train_data_pln_and_empty()))
print("no table publaynet word", len(get_train_data_and_word_pln_empty()))
print("no table publaynet latex", len(get_train_data_and_latex_pln_empty()))
print("no table publaynet latex and word", len(get_train_data_word_and_latex_pln_empty()))

word 73383
latex 187199
publaynet 85159
word publaynet 158542
latex publaynet 272358
no table publaynet 115159
no table publaynet word 188542
no table publaynet latex 302358
no table publaynet latex and word 375741


In [29]:
def get_test_data_word_2_tables():
    new_set =[]
    set_ = get_test_data_word()
    for i in range(len(set_)):
        if len(set_[i]["annotations"]) > 1:
            new_set.append(set_[i])
    return new_set

def get_test_data_latex_2_tables():
    new_set =[]
    set_ = get_test_data_latex()
    for i in range(len(set_)):
        if len(set_[i]["annotations"]) > 1:
            new_set.append(set_[i])
    return new_set

def get_test_data_publaynet_2_tables():
    new_set =[]
    set_ = get_test_data_publaynet()
    for i in range(len(set_)):
        if len(set_[i]["annotations"]) > 1:
            new_set.append(set_[i])
    return new_set

def get_test_data_word_1_table():
    new_set =[]
    set_ = get_test_data_word()
    for i in range(len(set_)):
        if len(set_[i]["annotations"]) == 1:
            new_set.append(set_[i])
    return new_set

def get_test_data_latex_1_table():
    new_set =[]
    set_ = get_test_data_latex()
    for i in range(len(set_)):
        if len(set_[i]["annotations"]) == 1:
            new_set.append(set_[i])
    return new_set

def get_test_data_publaynet_1_table():
    new_set =[]
    set_ = get_test_data_publaynet()
    for i in range(len(set_)):
        if len(set_[i]["annotations"]) == 1:
            new_set.append(set_[i])
    return new_set
    

In [23]:
train_set_name = "my_dataset_train2"
valid_set_name = "my_dataset_valid2"
test_set_name = "my_dataset_test2"

# Train set
DatasetCatalog.register(train_set_name, get_train_data_latex)
MetadataCatalog.get(train_set_name).set(thing_classes=["table"])
text_metadata_train = MetadataCatalog.get(train_set_name)

# Validation set
DatasetCatalog.register(valid_set_name, get_valid_data)
MetadataCatalog.get(valid_set_name).set(thing_classes=["table"])
text_metadata_valid = MetadataCatalog.get(valid_set_name)

# Test set
DatasetCatalog.register(test_set_name, get_test_data_publaynet)
MetadataCatalog.get(test_set_name).set(thing_classes=["table"])
text_metadata_test = MetadataCatalog.get(test_set_name)

AssertionError: Dataset 'my_dataset_train2' is already registered!

In [24]:
class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [36]:
"""
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3   # set the testing threshold for this model
cfg.DATASETS.TEST = (test_set_name, )
predictor = DefaultPredictor(cfg)
"""

type_ = "word-latex"

"""
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
#cfg.DATASETS.TRAIN = (train_set_name,)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR + "/" + type_, "/model_final.pth")
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.TEST.EVAL_PERIOD = 1000
CUDA_LAUNCH_BLOCKING = 1
predictor = DefaultPredictor(cfg)
"""
cfg = get_cfg()
cfg.merge_from_file("/data/rali5/Tmp/yockelle/TableBank/TableBank/output/"+ type_ +"/X101/config_"+ type_ +"_X101.yaml")
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/data/rali5/Tmp/yockelle/TableBank/TableBank/output/"+ type_ +"/X101/model_final_"+ type_ +"_X101.pth")
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.TEST.EVAL_PERIOD = 1000
CUDA_LAUNCH_BLOCKING = 1
predictor = DefaultPredictor(cfg)

The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std
  proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


In [37]:
def draw_pred_and_truth(img, truths, preds):
    for truth in truths:
        x_min = int(truth[0])
        y_min = int(truth[1])
        x_max = int(truth[2])
        y_max = int(truth[3])
        cv2.rectangle(img, (x_min, y_max), (x_max, y_min), (0,0,255), 2)
    for pred in preds:
        x_min = int(pred[0])
        y_min = int(pred[1])
        x_max = int(pred[2])
        y_max = int(pred[3])
        cv2.rectangle(img, (x_min, y_max), (x_max, y_min), (255,0,0), 1)

def draw_cv2(img, predictions):
    for pred in predictions:
        x_min, y_min, x_max, y_max = pred
        cv2.rectangle(img, (x_min, y_max), (x_max, y_min), (138,43,226), 1)


def combine_two_models(img, preds_detectron, preds_cv2, iou_max_threshold, does_print = True):
    new_predictions = []
    i = 0

    for pred_detectron in preds_detectron:
        # Try to match a cv2_preds to a detectron pred
        result = get_max_iou(np.array(preds_cv2), pred_detectron)

        if result is not None:
            iou_max = result[1]
            index = result[2]
            associated_cv2_pred = preds_cv2[index]
            x_min_cv2, y_min_cv2, x_max_cv2, y_max_cv2 = associated_cv2_pred
            x_min_detectron, y_min_detectron, x_max_detectron, y_max_detectron = pred_detectron
            area_cv2 = (x_max_cv2 - x_min_cv2) * (y_max_cv2 - y_min_cv2)
            area_detectron = (x_max_detectron - x_min_detectron) * (y_max_detectron - y_min_detectron)
            if iou_max>=iou_max_threshold:
                if area_detectron>area_cv2:
                    new_predictions.append(associated_cv2_pred) 
                else:
                    new_predictions.append(pred_detectron) 
            else:
                new_predictions.append(pred_detectron) 
        # if we get no match
        else:
            new_predictions.append(pred_detectron) 

        # Draw
        if does_print:
            x_min, y_min, x_max, y_max = new_predictions[i]
            cv2.rectangle(img, (int(x_min), int(y_max)), (int(x_max), int(y_min)), (0,255,0), 3)
            i += 1

    return np.array(new_predictions)


In [44]:
#test_data = get_test_data_word()
#test_data_publaynet = get_test_data_latex()
test_type_name = "latex"
is_show_image = False

threshold = 0.975

IOU = 0.9

precision_tab = []
recall_tab = []

sum_numerator = 0
sum_numerator_old = 0
sum_denominator_precision = 0
sum_denominator_recall = 0
sum_numerator_comb = 0
sum_numerator_old_comb = 0
sum_denominator_precision_comb = 0
sum_denominator_recall_comb = 0

count_bad_precision = 0

test_data = []
if "word" in test_type_name:
  test_data.extend(get_test_data_word_1_table())
if "latex" in test_type_name:
  test_data.extend(get_test_data_latex_2_tables())
if "publaynet" in test_type_name:
  test_data.extend(get_test_data_publaynet_1_table())

print(len(test_data))

for i in range(len(test_data)):
  #if i%1000 == 0:
  #  print(i)
  image_dict = test_data[i]

  try: 
    image_name = "/data/rali5/Tmp/yockelle/TableBank/" + image_dict["file_name"]  
    im = cv2.imread(image_name)
    does_shape_work = im.shape
  except:
    image_name = image_dict["file_name"]
    im = cv2.imread(image_name)

  outputs = predictor(im)

  predictions_detectron = outputs["instances"].pred_boxes.tensor.cpu().numpy()
  scores_detectron = outputs["instances"].scores.cpu().numpy()
  predictions_with_threshold = []
  for j, p in enumerate(predictions_detectron):
    if scores_detectron[j]>threshold:
      predictions_with_threshold.append(p)
  predictions_with_threshold = np.array(predictions_with_threshold)

  # Get the ground truth
  bbox_ground_truth = []
  truth = image_dict["annotations"]
  for t in truth:
      x_min = t["bbox"][0]
      y_min = t["bbox"][1]
      w = t["bbox"][2]
      h = t["bbox"][3]
      x_max = x_min + w
      y_max = y_min + h
      bbox_ground_truth.append([x_min, y_min, x_max, y_max])

  # get cv2 lines
  preds_cv2 = get_relevant_lines(image_name, return_bounding_box=True)
  # combine cv2 and detectron
  combined_predictions = combine_two_models(im, predictions_with_threshold, preds_cv2, IOU, does_print=is_show_image)

  # get metrics Detectron
  metric_result = metric_table_bank_union(bbox_ground_truth, predictions_with_threshold)

  if metric_result != None:
    numerator, denominator_precision, denominator_recall, old_way_area_union = metric_result

    sum_numerator += numerator
    sum_numerator_old += old_way_area_union
    sum_denominator_precision += denominator_precision
    sum_denominator_recall += denominator_recall

  # get metrics Combined
  metric_result_comb = metric_table_bank_union(bbox_ground_truth, combined_predictions)

  if metric_result_comb != None:
    numerator_comb, denominator_precision_comb, denominator_recall_comb, old_way_area_union_comb = metric_result_comb

    sum_numerator_comb += numerator_comb
    sum_numerator_old_comb += old_way_area_union_comb
    sum_denominator_precision_comb += denominator_precision_comb
    sum_denominator_recall_comb += denominator_recall_comb

  if is_show_image:
    #if numerator/denominator_precision <= 0.5:
    draw_pred_and_truth(im, bbox_ground_truth, predictions_with_threshold)
    draw_cv2(im, preds_cv2)
    plt.figure(figsize = (13,13))
    plt.imshow(im)
    plt.show()

precision = sum_numerator/sum_denominator_precision
recall = sum_numerator/sum_denominator_recall
f1 = (2 * precision * recall) / (precision + recall)
print("Precision ", precision)
print("Recall ", recall)
print("F1 ", f1)
print("\n")
#print("Number of bad precision: ", count_bad_precision)  


precision_comb = sum_numerator_comb/sum_denominator_precision_comb
recall_comb = sum_numerator_comb/sum_denominator_recall_comb
f1_comb = (2 * precision_comb * recall_comb) / (precision_comb + recall_comb)
print("Combining Detectron and cv2")
print("Precision ", precision_comb)
print("Recall ", recall_comb)
print("F1 ", f1_comb)
print("\n")

1156
Precision  0.985489717037302
Recall  0.9238066169024661
F1  0.9536517776785494


Combining Detectron and cv2
Precision  0.9870159572776256
Recall  0.9206244899821503
F1  0.95266491500347




### Measure with tablebank metric

In [ ]:
train_possibilities = ["latex", "word-latex"] #["word", "latex", "word-latex"]
test_possibilities = ["word", "latex", "publaynet", "word-latex", "word-publaynet" , "latex-publaynet", "word-latex-publaynet"]
threshold = 0.975

for train_type_name in train_possibilities:
  type_ = "/" + train_type_name + "/"
  train_type = train_type_name

  cfg = get_cfg()
  cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
  #cfg.DATASETS.TRAIN = (train_set_name,)
  cfg.DATASETS.TEST = ()
  cfg.DATALOADER.NUM_WORKERS = 4
  cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR + type_, "model_final.pth")
  cfg.SOLVER.IMS_PER_BATCH = 4
  cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
  cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = 0.5
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
  cfg.TEST.EVAL_PERIOD = 1000
  CUDA_LAUNCH_BLOCKING = 1
  predictor = DefaultPredictor(cfg)

  for test_type_name in test_possibilities:
    print("TRAIN TYPE: ", train_type_name)
    print("TEST TYPE: ", test_type_name)

    test_latex_predictions = {}
    test_latex_truth = {}

    test_data = []

    if "word" in test_type_name:
      test_data.extend(get_test_data_word())
    if "latex" in test_type_name:
      test_data.extend(get_test_data_latex())
    if "publaynet" in test_type_name:
      test_data.extend(get_test_data_publaynet())

    count_pred = 0
    count_truth = 0

    sum_numerator = 0
    sum_numerator_old = 0
    sum_denominator_precision = 0
    sum_denominator_recall = 0

    for index, image_dict in enumerate(test_data):
      one_image_pred = {}
      one_image_truth = {}

      try: 
        image_name = "/data/rali5/Tmp/yockelle/TableBank/" + image_dict["file_name"]  
        im = cv2.imread(image_name).shape
      except:
        image_name = image_dict["file_name"]
        im = cv2.imread(image_name).shape
      #image_name = "/data/rali5/Tmp/yockelle/PubLayNet/" + image_dict["file_name"]
      #image_name = image_dict["file_name"]
      only_name = image_dict["file_name"].split("/")[-1]

      truths_boxes = []
      preds_boxes = []

      problem = False

      # ADD TRUTH
      for annot in image_dict["annotations"]:
        p = annot["bbox"]
        x1 = p[0]
        y1 = p[1]
        w = p[2]
        h = p[3]
        x2 = p[0] + w
        y2 = p[1] + h
        #if y1 == y2 or y2<y1:
        #  problem = True
        #  break

        #if not(problem):
        count_truth += 1
        truths_boxes.append([x1,y1,x2,y2])
      
      # ADD PREDS
      im = cv2.imread(image_name)
      outputs = predictor(im)
      predictions_detectron = outputs["instances"].pred_boxes.tensor.cpu().numpy()
      scores_detectron = outputs["instances"].scores.cpu().numpy()

      for j, p in enumerate(predictions_detectron):
        if scores_detectron[j]>threshold:
          x1 = p[0]
          y1 = p[1]
          x2 = p[2]
          y2 = p[3]
          w = x2-x1
          h = y2-y1
          count_pred += 1
          preds_boxes.append([x1,y1,x2,y2])

      result = metric_table_bank_union(np.array(truths_boxes), np.array(preds_boxes))
      numerator, denominator_precision, denominator_recall, old_way_area_union = result

      sum_numerator += numerator
      sum_numerator_old += old_way_area_union
      sum_denominator_precision += denominator_precision
      sum_denominator_recall += denominator_recall

    precision = sum_numerator/sum_denominator_precision
    recall = sum_numerator/sum_denominator_recall
    f1 = (2 * precision * recall) / (precision + recall)

    print("Precision ", precision)
    print("Recall ", recall)
    print("F1 ", f1)
    print("-----------------------------------------------------")

### Measure for all metrics

In [ ]:
train_possibilities = ["word"]
test_possibilities = ["word", "latex", "word-latex"] #"word-publaynet" , "latex-publaynet", "word-latex-publaynet"]
#test_possibilities = ["publaynet"]

for train_type_name in train_possibilities:

  type_ = "/" + train_type_name + "/"
  train_type = train_type_name

  cfg = get_cfg()
  cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
  #cfg.DATASETS.TRAIN = (train_set_name,)
  cfg.DATASETS.TEST = ()
  cfg.DATALOADER.NUM_WORKERS = 4
  cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR + type_, "model_final.pth")
  cfg.SOLVER.IMS_PER_BATCH = 4
  cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
  cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = 0.5
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
  cfg.TEST.EVAL_PERIOD = 1000
  CUDA_LAUNCH_BLOCKING = 1
  predictor = DefaultPredictor(cfg)

  for test_type_name in test_possibilities:
    print(test_type_name)

    test_latex_predictions = {}
    test_latex_truth = {}

    test_data = []

    if "word" in test_type_name:
      test_data.extend(get_test_data_word())
    if "latex" in test_type_name:
      test_data.extend(get_test_data_latex())
    if "publaynet" in test_type_name:
      test_data.extend(get_test_data_publaynet())

    test_type = test_type_name
    
    count_pred = 0
    count_truth = 0

    all_image_preds = []
    all_annotations_preds = []

    all_image_truth = []
    all_annotations_truth = []

    sum_numerator = 0
    sum_numerator_old = 0
    sum_denominator_precision = 0
    sum_denominator_recall = 0

    for index, image_dict in enumerate(test_data):
      one_image_pred = {}
      one_image_truth = {}
      
      #if index%500 == 0:
      #  print(index)

      try: 
        image_name = "/data/rali5/Tmp/yockelle/TableBank/" + image_dict["file_name"]  
        im = cv2.imread(image_name).shape
      except:
        image_name = image_dict["file_name"]
        im = cv2.imread(image_name).shape
      #image_name = "/data/rali5/Tmp/yockelle/PubLayNet/" + image_dict["file_name"]
      #image_name = image_dict["file_name"]
      only_name = image_dict["file_name"].split("/")[-1]

      truths_boxes = []
      preds_boxes = []

      problem = False

      # ADD TRUTH
      for annot in image_dict["annotations"]:
        #if scores_detectron[j]>threshold:
        p = annot["bbox"]
        x1 = p[0]
        y1 = p[1]
        w = p[2]
        h = p[3]
        x2 = p[0] + w
        y2 = p[1] + h
        if y1 == y2 or y2<y1:
          #print("here")
          #print(count_truth)
          problem = True
          break

        if not(problem):
          
          one_annotations = {}
          one_annotations["id"] = count_truth
          one_annotations["segmentation"] = []
          one_annotations["area"] = float(w * h)
          one_annotations["iscrowd"] = 0
          one_annotations["ignore"] = 0
          one_annotations["image_id"] = index
          one_annotations["bbox"] = [float(pp) for pp in p]
          one_annotations["category_id"] = 0

          all_annotations_truth.append(one_annotations)
          count_truth += 1

          truths_boxes.append([x1,y1,x2,y2])

      if problem:
        continue

      if problem:
        print("wtf")

      one_image_truth["id"] = index
      one_image_truth["file_name"] = only_name
      one_image_truth["width"] = image_dict["width"]
      one_image_truth["height"] = image_dict["height"]

      all_image_truth.append(one_image_truth)
      

      # ADD PREDS
      im = cv2.imread(image_name)
      outputs = predictor(im)
      predictions_detectron = outputs["instances"].pred_boxes.tensor.cpu().numpy()
      scores_detectron = outputs["instances"].scores.cpu().numpy()

      for j, p in enumerate(predictions_detectron):
        #if scores_detectron[j]>threshold:
        x1 = p[0]
        y1 = p[1]
        x2 = p[2]
        y2 = p[3]
        w = x2-x1
        h = y2-y1
        #x2 = p[0] + w
        #y2 = p[1] + h
        
        one_annotations = {}
        one_annotations["id"] = count_pred
        one_annotations["segmentation"] = []
        one_annotations["area"] = float(w * h)
        one_annotations["iscrowd"] = 0
        one_annotations["ignore"] = 0
        one_annotations["image_id"] = index
        one_annotations["bbox"] = [float(pp) for pp in [x1,y1,w,h]]
        one_annotations["category_id"] = 0
        one_annotations["score"] = float(scores_detectron[j])

        all_annotations_preds.append(one_annotations)
        count_pred += 1

        preds_boxes.append([x1,y1,x2,y2])

      one_image_pred["id"] = index
      one_image_pred["file_name"] = only_name
      one_image_pred["width"] = image_dict["width"]
      one_image_pred["height"] = image_dict["height"]

      all_image_preds.append(one_image_pred)

      result = metric_table_bank_union(np.array(truths_boxes), np.array(preds_boxes))
      numerator, denominator_precision, denominator_recall, old_way_area_union = result

      sum_numerator += numerator
      sum_numerator_old += old_way_area_union
      sum_denominator_precision += denominator_precision
      sum_denominator_recall += denominator_recall


    # ADD TRUTHS
    test_latex_truth["images"] = all_image_truth
    test_latex_truth["annotations"] = all_annotations_truth
    test_latex_truth["categories"] = [{"id": 0, "name": "table", "supercategory": "none"}]

    path_truth = "prediction_and_truth_my_models/train_"+train_type+"/annotations_test_"+test_type+"/truth"
    if not path.exists(path_truth):
      os.makedirs(path_truth)
    out_file = open(path_truth + "/test_"+test_type+"_truth.json", "w")
    json.dump(test_latex_truth, out_file, indent = 6)
    out_file.close()
      
    # ADD PREDS
    test_latex_predictions["images"] = all_image_preds
    test_latex_predictions["annotations"] = all_annotations_preds
    test_latex_predictions["categories"] = [{"id": 0, "name": "table", "supercategory": "none"}]

    path_pred = "prediction_and_truth_my_models/train_"+train_type+"/annotations_test_"+test_type+"/preds"
    if not path.exists(path_pred):
      os.makedirs(path_pred)
    out_file = open(path_pred + "/test_"+test_type+"_preds.json", "w")
    json.dump(test_latex_predictions, out_file, indent = 6)
    out_file.close()

    precision = sum_numerator/sum_denominator_precision
    recall = sum_numerator/sum_denominator_recall
    f1 = (2 * precision * recall) / (precision + recall)

    print("First")
    print("Precision ", precision)
    print("Recall ", recall)
    print("F1 ", f1)